<a href="https://colab.research.google.com/github/BrystofKlazek/RAD/blob/main/code/01RAD_Ex11_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01RAD Exercise 11 - Hands on

Authors: Your names here  
Date: 2025-12-10  

---

## Task Description

The dataset is based on the **House Sales in King County, USA** dataset, which can be found, for example, on kaggle.com or in the `moderndive` library under the name `house_prices`. The original dataset contains house sale prices in the King County area, which includes Seattle, and the data was collected between May 2014 and May 2015. For our purposes, several variables have been removed, and the dataset has been significantly reduced and slightly modified.

The dataset has already been split into three parts and modified, all of which will be used progressively throughout this assignment.

---

## Variables Description

The dataset contains the following 18 variables, and our goal is to explore the influence of 12 of them on the target variable `price`.

| Feature         | Description                                           |
|------------------|-------------------------------------------------------|
| `id`            | Unique identifier for a house                         |
| `price`         | Sale price (prediction target)                        |
| `bedrooms`      | Number of bedrooms                                    |
| `bathrooms`     | Number of bathrooms                                   |
| `sqft_living`   | Square footage of the home                            |
| `sqft_lot`      | Square footage of the lot                             |
| `floors`        | Total number of floors (levels) in the house          |
| `waterfront`    | Whether the house has a waterfront view               |
| `view`          | Number of times the house has been viewed             |
| `condition`     | Overall condition of the house                        |
| `grade`         | Overall grade given to the housing unit               |
| `sqft_above`    | Square footage of the house apart from the basement   |
| `sqft_basement` | Square footage of the basement                        |
| `yr_built`      | Year the house was built                              |
| `yr_renovated`  | Year when the house was renovated                     |
| `sqft_living15` | Living room area in 2015 (after renovations)          |
| `sqft_lot15`    | Lot size in 2015 (after renovations)                  |
| `split`         | Splitting variable with train, test, and validation samples |

---


In [ ]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

import matplotlib.pyplot as plt

# Load the dataset
url = "https://raw.githubusercontent.com/francji1/01RAD/main/data/01RAD_2024_house.csv"
house_rad = pd.read_csv(url)

# Display the first few rows of the dataset
house_rad.head()

# Convenience splits for later
train = house_rad.query('split == "train"').copy()
test  = house_rad.query('split == "test"').copy()
val   = house_rad.query('split == "validation"').copy()




---

## Exploratory and Graphical Analysis

### Question 1

Verify the dimensions of the dataset, the types of individual variables, and summarize the basic descriptive statistics of all variables. Plot a histogram and a density estimate for the target variable `price`. Can anything be inferred for future analysis?

---

### Question 2

Are all variables usable for analysis and prediction of house prices? If the data contains missing values (or strange or nonsensical observations), can they be imputed (corrected), or must they be removed from the dataset?

---

### Question 3

For the selected variables (`price`, `sqft_living`, `grade`, `yr_built`), verify whether the split into train, test, and validation datasets was random. That is, do these variables have approximately the same distributions across the train, test, and validation groups?

---


Consider to transform price (e.g. log‑price) and key size variables if distributions are highly skewed and compare residual diagnostics for transformed vs untransformed models later.

In [ ]:
# Dimensions
print(house_rad.shape)
print(house_rad.apply(len).unique()) #This comes out fine - all cols are the same length

print(house_rad.dtypes)

print(house_rad.describe().T)

print(house_rad['price'].describe())

low_price = house_rad[house_rad['price']==0]
print(low_price)
house_rad['log_price'] = np.log(house_rad['price'])

plt.figure()
plt.hist(house_rad['log_price'], bins=50)
plt.title("log_rice histogram")
plt.xlabel("log_price")
plt.ylabel("count")
plt.show()



In [ ]:
import numpy as np

house_rad = house_rad.copy()

house_rad['house_age'] = np.where(
    house_rad['yr_renovated'] != 0,
    2015 - house_rad['yr_renovated'],       # renovated → age since renovation
    2015 - house_rad['yr_built']            # not renovated → age since built
)

house_rad = house_rad.drop(columns=['yr_renovated'])
house_rad = house_rad.drop(index=[148, 48])
house_rad = house_rad.query("grade <= 13").copy()


In [ ]:
house_rad['split'].value_counts()


In [ ]:
vars_q3 = ['price', 'sqft_living', 'grade', 'yr_built', 'log_price']

summary_by_split = (
    house_rad
    .groupby('split')[vars_q3]
    .describe()
    .round(2)
)

summary_by_split


In [ ]:
train = house_rad.query('split == "train"')
test  = house_rad.query('split == "test"')
val   = house_rad.query('split == "validation"')

for v in vars_q3:
    plt.figure()
    plt.hist(train[v].dropna(), bins=40, alpha=0.4, label='train')
    plt.hist(test[v].dropna(),  bins=40, alpha=0.4, label='test')
    plt.hist(val[v].dropna(),   bins=40, alpha=0.4, label='validation')
    plt.title(f"{v} distribution by split")
    plt.xlabel(v)
    plt.ylabel("count")
    plt.legend()
    plt.show()



## Linear Model (Use Only Training Data, i.e., split == "train")

### Question 4

Calculate the correlations between the regressors and visualize them. Also, compute the condition number (Kappa) and the variance inflation factor (VIF). If multicollinearity is present, decide which variables to use and justify your choices.

---

### Question 5

Using only the training data (split == "train") and all selected variables, find a suitable linear regression model that best predicts the target variable `price`, i.e., minimizes the mean squared error (MSE). Compare the VIF and Kappa values of the final model to those of the original regressor matrix.

---

### Question 6

For your selected model from the previous question, calculate the relevant influence measures. Provide the `id` for the top 20 observations with the highest values of DIFFITS and DFBetas, the highest leverage (hat values), and the highest Cook’s distance (i.e., 3 sets of 20 values). Which observations do you consider influential or outliers?

---

### Question 7

Validate the model graphically using residual plots (Residual vs. Fitted, QQ-plot, Cook’s distance, leverage, etc.). Based on this and the previous question, did you identify any suspicious observations in the data that might have resulted from dataset adjustments? Would you recommend removing these observations from the data?

---


In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import patsy

train = house_rad.query('split == "train"').copy()
test  = house_rad.query('split == "test"').copy()
val   = house_rad.query('split == "validation"').copy()

exclude = {'price', 'split', 'id', 'Unnamed: 0', 'log_price'}
exclude = {c for c in exclude if c in train.columns}

num_cols = train.select_dtypes(include='number').columns
regressors = [c for c in num_cols if c not in exclude]

regressors

formula_full = "log_price ~ " + " + ".join(regressors)
formula_full



In [ ]:
corr_reg = train[regressors].corr()

corr_reg.round(3).head()

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(corr_reg, annot=True, fmt=".2f", cmap="coolwarm", center=0)
plt.title("Correlation matrix of regressors (train)")
plt.show()


In [ ]:
y, X = patsy.dmatrices(formula_full, data=train, return_type='dataframe')

# Condition number of X (includes intercept)
kappa = np.linalg.cond(X)
kappa

In [ ]:
#STANDARDIZED FOR BETTER RESULTS
X_no_intercept = X.drop(columns=['Intercept'], errors='ignore')

X_std = (X_no_intercept - X_no_intercept.mean()) / X_no_intercept.std(ddof=0)

kappa_std = np.linalg.cond(X_std)
kappa_std


In [ ]:
# Compute VIF for each column in X
vif_df = pd.DataFrame({
    'term': X.columns,
    'VIF': [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
}).sort_values('VIF', ascending=False)

vif_df


In [ ]:
train = house_rad.query('split == "train"').copy()
test  = house_rad.query('split == "test"').copy()
val   = house_rad.query('split == "validation"').copy()


exclude = {'price', 'split', 'id', 'Unnamed: 0', 'log_price'}
exclude = {c for c in exclude if c in train.columns}

num_cols = train.select_dtypes(include='number').columns
regressors = [c for c in num_cols if c not in exclude]

regressors

formula_original = "log_price ~ " + " + ".join(regressors)
formula_original

exclude = {'price', 'split', 'id', 'Unnamed: 0', 'sqft_above', 'sqft_lot', 'sqft_living', 'yr_renovated', 'yr_built', 'log_price'}
exclude = {c for c in exclude if c in train.columns}

num_cols = train.select_dtypes(include='number').columns
regressors = [c for c in num_cols if c not in exclude]

regressors

formula_full = "log_price ~ " + " + ".join(regressors)
formula_full



def mse(y_true, y_pred):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return np.mean((y_true - y_pred)**2)

def kappa_of_formula(formula, data):
    y, X = patsy.dmatrices(formula, data=data, return_type='dataframe')
    kappa_raw = np.linalg.cond(X)

    # scaled version (more meaningful for multicollinearity)
    X_no_int = X.drop(columns=['Intercept'], errors='ignore')
    X_std = (X_no_int - X_no_int.mean()) / X_no_int.std(ddof=0)
    kappa_std = np.linalg.cond(X_std)

    return kappa_raw, kappa_std, X

def vif_of_X(X):
    vif_df = pd.DataFrame({
        'term': X.columns,
        'VIF': [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    }).sort_values('VIF', ascending=False)
    return vif_df


In [ ]:
# 1) Fit both models on TRAIN
m_orig  = smf.ols(formula_original, data=train).fit()
m_final = smf.ols(formula_full,     data=train).fit()

# 2) Training MSE
mse_orig  = mse(train['log_price'], m_orig.predict(train))
mse_final = mse(train['log_price'], m_final.predict(train))

rmse_orig  = np.sqrt(mse_orig)
rmse_final = np.sqrt(mse_final)

# 3) Kappa + VIF for each

# Original
kappa_orig_raw, kappa_orig_std, X_orig = kappa_of_formula(formula_original, train)
vif_orig = vif_of_X(X_orig)
max_vif_orig = vif_orig.query("term != 'Intercept'")['VIF'].max()

# Final
kappa_final_raw, kappa_final_std, X_final = kappa_of_formula(formula_full, train)
vif_final = vif_of_X(X_final)
max_vif_final = vif_final.query("term != 'Intercept'")['VIF'].max()

# 4) Side-by-side comparison
comparison = pd.DataFrame([
    {
        'model': 'original',
        'train_mse': mse_orig,
        'train_rmse': rmse_orig,
        'kappa_raw': kappa_orig_raw,
        'kappa_std': kappa_orig_std,
        'max_vif_non_intercept': max_vif_orig
    },
    {
        'model': 'final',
        'train_mse': mse_final,
        'train_rmse': rmse_final,
        'kappa_raw': kappa_final_raw,
        'kappa_std': kappa_final_std,
        'max_vif_non_intercept': max_vif_final
    }
])

comparison


In [ ]:
from statsmodels.stats.outliers_influence import OLSInfluence

# Influence object
infl = m_final.get_influence()

# DFFITS
dffits, dffits_p = infl.dffits

# DFBetas: array shape (n_obs, k_params)
dfbetas = infl.dfbetas

# Leverage (hat values)
hat = infl.hat_matrix_diag

# Cook's distance
cooks, cooks_p = infl.cooks_distance



# Build a diagnostic DataFrame indexed like train
diag = train[['id']].copy()
diag['dffits'] = dffits
diag['hat'] = hat
diag['cooks'] = cooks

# Max absolute DFBetas over all coefficients for each observation
diag['max_dfbeta'] = np.abs(dfbetas).max(axis=1)

# Absolute values for ranking
diag['abs_dffits'] = np.abs(diag['dffits'])
diag['abs_max_dfbeta'] = np.abs(diag['max_dfbeta'])


In [ ]:
top20_dffits = diag.nlargest(20, 'abs_dffits')[['id', 'dffits']]
top20_dfbeta = diag.nlargest(20, 'abs_max_dfbeta')[['id', 'max_dfbeta']]
top20_hat    = diag.nlargest(20, 'hat')[['id', 'hat']]
top20_cooks  = diag.nlargest(20, 'cooks')[['id', 'cooks']]

print("Top 20 by |DFFITS|:")
print(top20_dffits)

print("\nTop 20 by max |DFBeta|:")
print(top20_dfbeta)

print("\nTop 20 by leverage (hat):")
print(top20_hat)

print("\nTop 20 by Cook's distance:")
print(top20_cooks)


In [ ]:
n = m_final.nobs
p = m_final.df_model + 1

# Rules of thumb
dffits_cut = 2 * np.sqrt(p / n)
cooks_cut  = 4 / n
hat_cut    = 2 * p / n     # or 3*p/n
dfbeta_cut = 2 / np.sqrt(n)

dffits_cut, cooks_cut, hat_cut, dfbeta_cut


In [ ]:
# 1) Flag observations
diag['flag_dffits'] = diag['abs_dffits']       > dffits_cut
diag['flag_cooks']  = diag['cooks']            > cooks_cut
diag['flag_hat']    = diag['hat']              > hat_cut
diag['flag_dfbeta'] = diag['abs_max_dfbeta']   > dfbeta_cut

flag_cols = ['flag_dffits','flag_cooks','flag_hat','flag_dfbeta']

# total number of flags per observation
diag['n_flags'] = diag[flag_cols].sum(axis=1)

# 2) Rows suspicious in at least one sense (or use >=2 if you want stricter)
suspects = diag[diag['n_flags'] >= 1].copy()

# sort by how many flags, then Cook's D, then |DFFITS|
suspects = suspects.sort_values(
    ['n_flags','cooks','abs_dffits'],
    ascending=[False, False, False]
)

suspects.head(30)[['id','dffits','max_dfbeta','hat','cooks','n_flags']]

suspects = diag[diag['n_flags'] >= 3].copy()

# 3) Inspect the underlying data for those suspicious ids
suspects_ids = suspects['id'].tolist()

cols_to_inspect = [
    'id', 'price', 'sqft_living', 'sqft_lot',
    'bedrooms', 'bathrooms', 'grade',
    'house_age', 'view', 'waterfront'
]

train.loc[train['id'].isin(suspects_ids), cols_to_inspect] \
     .sort_values('price', ascending=False)


In [ ]:
m_final = smf.ols(formula_full,     data=train).fit()


# Residuals and fitted values from the final model
resid   = m_final.resid
fitted  = m_final.fittedvalues

# 1) Residuals vs Fitted
plt.figure(figsize=(6,4))
plt.scatter(fitted, resid, alpha=0.5)
plt.axhline(0, color='black', linewidth=1)
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (train)")
plt.tight_layout()
plt.show()


infl = m_final.get_influence()
resid_stud = infl.resid_studentized_internal

fig = plt.figure(figsize=(6,4))
sm.qqplot(resid_stud, line='45', ax=fig.add_subplot(111))
plt.title("QQ-plot of studentized residuals (train)")
plt.tight_layout()
plt.show()



infl = m_final.get_influence()
leverage = infl.hat_matrix_diag
stud_resid = infl.resid_studentized_external
cooks = infl.cooks_distance[0]

inf_df = pd.DataFrame({
    'id': train['id'].values,
    'leverage': leverage,
    'stud_resid': stud_resid,
    'cooks': cooks
})

plt.figure(figsize=(7,5))
ax = sns.scatterplot(
    data=inf_df,
    x='leverage',
    y='stud_resid',
    size='cooks',
    hue='cooks',
    palette='viridis',
    alpha=0.7,
    legend=False,
)

# mark a leverage cutoff
n = m_final.nobs
p = m_final.df_model + 1
hat_cut = 2 * p / n
ax.axvline(hat_cut, color='red', linestyle='--')

ax.set_xlabel("Leverage")
ax.set_ylabel("Studentized residuals")
ax.set_title("Influence plot (size/color = Cook's distance)")
plt.tight_layout()
plt.show()



Using the influence diagnostics and residual plots from Questions 6 and 7, I inspected the underlying covariates of the most influential observations. Most of the high-price points correspond to large, high-grade houses with big lots and sometimes views or waterfront, so they appear to be genuine luxury properties.

However, some cases stand out as suspicious: in particular ids 100, 200, 300 and 109 have extremely high prices (4.45–8.5 million) despite modest living area, small lots, low/medium grades, and no view or waterfront, and ids 215 and 119 have extremely low prices (84 000 and 28 500) for fairly ordinary houses. These combinations are inconsistent with the rest of the dataset and likely reflect dataset adjustments or data-entry errors rather than genuine sales.

Based on this inspection, I would treat ids 100, 200, 300, 109, 215 and 119 as suspicious outliers caused by data issues and I would recommend removing them before fitting the final model. The remaining influential observations look like valid but extreme properties, so I keep them in the data and simply note that they have a noticeable influence on the fitted regression.

In [ ]:
# IDs to drop (suspicious / likely bad)
bad_ids = [100, 200, 300, 109, 215, 119]

# Drop them from the full dataset
house_rad = house_rad[~house_rad['id'].isin(bad_ids)].copy()

# Recreate splits so train/test/val reflect the change
train = house_rad.query('split == "train"').copy()
test  = house_rad.query('split == "test"').copy()
val   = house_rad.query('split == "validation"').copy()

# Optional: quick check they are really gone
house_rad[house_rad['id'].isin(bad_ids)]


In [ ]:

m_final = smf.ols(formula_full,     data=train).fit()

# Residuals and fitted values from the final model
resid   = m_final.resid
fitted  = m_final.fittedvalues


# Influence object
infl = m_final.get_influence()

# DFFITS
dffits, dffits_p = infl.dffits

# DFBetas: array shape (n_obs, k_params)
dfbetas = infl.dfbetas

# Leverage (hat values)
hat = infl.hat_matrix_diag

# Cook's distance
cooks, cooks_p = infl.cooks_distance

# 1) Flag observations
diag['flag_dffits'] = diag['abs_dffits']       > dffits_cut
diag['flag_cooks']  = diag['cooks']            > cooks_cut
diag['flag_hat']    = diag['hat']              > hat_cut
diag['flag_dfbeta'] = diag['abs_max_dfbeta']   > dfbeta_cut

flag_cols = ['flag_dffits','flag_cooks','flag_hat','flag_dfbeta']

# total number of flags per observation
diag['n_flags'] = diag[flag_cols].sum(axis=1)

# 2) Rows suspicious in at least one sense (or use >=2 if you want stricter)
suspects = diag[diag['n_flags'] >= 1].copy()

# sort by how many flags, then Cook's D, then |DFFITS|
suspects = suspects.sort_values(
    ['n_flags','cooks','abs_dffits'],
    ascending=[False, False, False]
)

suspects.head(30)[['id','dffits','max_dfbeta','hat','cooks','n_flags']]

suspects = diag[diag['n_flags'] >= 3].copy()

# 3) Inspect the underlying data for those suspicious ids
suspects_ids = suspects['id'].tolist()

cols_to_inspect = [
    'id', 'price', 'sqft_living', 'sqft_lot',
    'bedrooms', 'bathrooms', 'grade',
    'house_age', 'view', 'waterfront'
]

train.loc[train['id'].isin(suspects_ids), cols_to_inspect] \
     .sort_values('price', ascending=False)




# Build a diagnostic DataFrame indexed like train
diag = train[['id']].copy()
diag['dffits'] = dffits
diag['hat'] = hat
diag['cooks'] = cooks

# Max absolute DFBetas over all coefficients for each observation
diag['max_dfbeta'] = np.abs(dfbetas).max(axis=1)

# Absolute values for ranking
diag['abs_dffits'] = np.abs(diag['dffits'])
diag['abs_max_dfbeta'] = np.abs(diag['max_dfbeta'])


# 1) Residuals vs Fitted
plt.figure(figsize=(6,4))
plt.scatter(fitted, resid, alpha=0.5)
plt.axhline(0, color='black', linewidth=1)
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (train)")
plt.tight_layout()
plt.show()

# studentized internal residuals
infl = m_final.get_influence()
resid_stud = infl.resid_studentized_internal

fig = plt.figure(figsize=(6,4))
sm.qqplot(resid_stud, line='45', ax=fig.add_subplot(111))
plt.title("QQ-plot of studentized residuals (train)")
plt.tight_layout()
plt.show()



infl = m_final.get_influence()
leverage = infl.hat_matrix_diag
stud_resid = infl.resid_studentized_external
cooks = infl.cooks_distance[0]

inf_df = pd.DataFrame({
    'id': train['id'].values,
    'leverage': leverage,
    'stud_resid': stud_resid,
    'cooks': cooks
})

plt.figure(figsize=(7,5))
ax = sns.scatterplot(
    data=inf_df,
    x='leverage',
    y='stud_resid',
    size='cooks',
    hue='cooks',
    palette='viridis',
    alpha=0.7,
    legend=False,
)

# mark a leverage cutoff
n = m_final.nobs
p = m_final.df_model + 1
hat_cut = 2 * p / n
ax.axvline(hat_cut, color='red', linestyle='--')

ax.set_xlabel("Leverage")
ax.set_ylabel("Studentized residuals")
ax.set_title("Influence plot (size/color = Cook's distance)")
plt.tight_layout()
plt.show()




## Robust Approach

### Question 8

If you decided to remove any observations from the dataset, work with the filtered dataset, retrain the model from Question 5, and calculate the $R^2$ statistic and MSE on both the training and test data (split == "test").

---

### Question 9

Estimate the regression coefficients using a robust Total Least Squares (TLS), on both the filtered dataset (after removing any observations, if applicable) and the original full dataset. Compare the results and discuss any differences in the estimated coefficients and model performance. What insights can you draw about the impact of filtering observations on model robustness?


---
### Question 10

Select the final model and compare the MSE and $R^2$ on the training, test, and validation datasets. What do these values suggest about the quality of the model and potential overfitting? Is your model suitable for predicting house prices in the King County area? If so, does this prediction have any limitations?



In [ ]:
from sklearn.metrics import  r2_score

target = 'log_price'

# predictions
pred_train = m_final.predict(train)
pred_test  = m_final.predict(test)

# metrics
mse_train = mse(train[target], pred_train)
mse_test  = mse(test[target],  pred_test)

r2_train  = r2_score(train[target], pred_train)
r2_test   = r2_score(test[target],  pred_test)

metrics_q8 = pd.DataFrame([
    {'split': 'train', 'MSE': mse_train, 'RMSE': np.sqrt(mse_train), 'R2': r2_train},
    {'split': 'test',  'MSE': mse_test,  'RMSE': np.sqrt(mse_test),  'R2': r2_test},
])

metrics_q8


In [ ]:
import patsy
import numpy as np
import pandas as pd

def tls_fit_from_formula(formula, data):

    y, X = patsy.dmatrices(formula, data=data, return_type='dataframe')
    y = np.asarray(y).ravel()
    X_mat = np.asarray(X)

    # build augmented matrix [X | y]
    Z = np.column_stack([X_mat, y])

    # TLS via SVD: last right-singular vector
    U, S, Vt = np.linalg.svd(Z, full_matrices=False)
    v = Vt[-1, :]             # last row

    v_x = v[:-1]
    v_y = v[-1]
    beta_tls = -v_x / v_y     # TLS coefficients

    beta_tls = pd.Series(beta_tls, index=X.columns)
    return beta_tls, X, y


In [ ]:
# TLS on filtered train data
beta_tls_filt, X_filt, y_filt = tls_fit_from_formula(formula_full, train)

# Compare with OLS coefficients of the same model
coef_ols_filt = m_final.params

coef_compare_filt = pd.DataFrame({
    'OLS_filtered': coef_ols_filt,
    'TLS_filtered': beta_tls_filt
})
coef_compare_filt


In [ ]:
y_pred_tls_filt = X_filt @ beta_tls_filt.values

mse_tls_filt = mse(y_filt, y_pred_tls_filt)
r2_tls_filt  = r2_score(y_filt, y_pred_tls_filt)
mse_tls_filt, r2_tls_filt


In [ ]:
house_rad_full= pd.read_csv(url)

house_rad_full['log_price'] = np.log(house_rad_full['price'])


house_rad_full['house_age'] = np.where(
    house_rad_full['yr_renovated'] != 0,
    2015 - house_rad_full['yr_renovated'],       # renovated → age since renovation
    2015 - house_rad_full['yr_built']            # not renovated → age since built
)


train_full = house_rad_full.query('split == "train"').copy()

# OLS on full data
m_ols_full = smf.ols(formula_full, data=train_full).fit()

# TLS on full data
beta_tls_full, X_full, y_full = tls_fit_from_formula(formula_full, train_full)
coef_ols_full = m_ols_full.params

coef_compare_full = pd.DataFrame({
    'OLS_full': coef_ols_full,
    'TLS_full': beta_tls_full
})
coef_compare_full

y_pred_tls_full = X_full @ beta_tls_full.values
mse_tls_full = mse(y_full, y_pred_tls_full)
r2_tls_full  = r2_score(y_full, y_pred_tls_full)
mse_tls_full, r2_tls_full


In [ ]:
splits = {
    'train': train,
    'test':  test,
    'validation': val
}

rows = []
for name, df in splits.items():
    y_true = df[target]
    y_pred = m_final.predict(df)
    mse_   = mse(y_true, y_pred)
    r2_    = r2_score(y_true, y_pred)
    rows.append({
        'split': name,
        'MSE': mse_,
        'RMSE': np.sqrt(mse_),
        'R2': r2_
    })

metrics_q10 = pd.DataFrame(rows)
metrics_q10


---

## Machine Learning Approach


### Question 11
Apply machine learning-based linear regression methods, such as Ridge Regression, Lasso, or Elastic Net, to the dataset. Use the train-test split to evaluate model performance and focus on feature selection. Identify the most relevant features based on these methods and compare how the selected features impact the model's predictive performance. Discuss how regularization affects feature selection and the trade-offs between bias and variance in the context of house price prediction. Additionally, evaluate the stability of selected features across different methods and provide recommendations for choosing the optimal feature set.

Use
* Standardization of regressors (and possibly use of pipelines).
* Hyperparameter tuning via cross‑validation (e.g. grid search over λ/α).
* Compare OLS vs ridge vs lasso on the same train/test split.


In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# parse RHS of formula_full: "price ~ x1 + x2 + ..." -> ["x1", "x2", ...]
rhs = formula_full.split('~', 1)[1]
predictors = [s.strip() for s in rhs.split('+')]

X_train = train[predictors].values
y_train = train['price'].values

X_test  = test[predictors].values
y_test  = test['price'].values


In [ ]:
def eval_model(name, model, X_train, y_train, X_test, y_test):
    y_pred_train = model.predict(X_train)
    y_pred_test  = model.predict(X_test)

    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test  = mean_squared_error(y_test,  y_pred_test)
    r2_train  = r2_score(y_train, y_pred_train)
    r2_test   = r2_score(y_test,  y_pred_test)

    return {
        'model': name,
        'train_MSE': mse_train,
        'train_RMSE': np.sqrt(mse_train),
        'train_R2': r2_train,
        'test_MSE': mse_test,
        'test_RMSE': np.sqrt(mse_test),
        'test_R2': r2_test
    }
